In [1]:
import os
from datetime import datetime
import csv

import pandas as pd
from sympy import false

from landbosse.excelio import XlsxSerialManagerRunner
from landbosse.excelio import XlsxParallelManagerRunner
from landbosse.excelio import XlsxGenerator
from landbosse.excelio import XlsxValidator
from landbosse.excelio import CsvGenerator

# LandBOSSE, small utility functions
from landbosse.excelio import XlsxFileOperations
set_path = True
if set_path:
    input_output_path = '~/Desktop/'
    os.environ["LANDBOSSE_INPUT_DIR"] = input_output_path
else:
    input_output_path = os.path.dirname(__file__)
    os.environ["LANDBOSSE_INPUT_DIR"] = input_output_path


if __name__ == '__main__':
    # Print start timestamp
    print(f'>>>>>>>> Begin run {datetime.now()} <<<<<<<<<<')

    # The file_ops object handles file names for input and output data.
    file_ops = XlsxFileOperations()

    # If run_parallel is True, an XlsxParallelManagerRunner will calculate the
    # projects in parallel. This takes advantage of multicore architecture
    # available on most hardware.
    #
    # If run_parallel is False, XlsxSerialManagerRunner will calculate projects
    # serially. This is much slower so running in parallel is preferred
    # unless there is a good reason to run serially. One such reason is using a
    # debugger which can slow down when it is being used to debug multiple
    # processes.

    run_parallel = False
    manager_runner = XlsxParallelManagerRunner(file_ops) if run_parallel else XlsxSerialManagerRunner(file_ops)

    # project_xlsx is the absolute path of the project_list.xlsx
    projects_xlsx = os.path.join(file_ops.landbosse_input_dir(), 'project_list.xlsx')

    # Should we enable the scaling study? Just look for the scaling study
    # option. The first three elements of the returned tuple aren't necessary
    # to determine if the scaling study is enabled.
    # Switch to either validation or non validation producing code.
    input_path, output_path, validation_enabled, enable_scaling_study = file_ops.get_input_output_paths_from_argv_or_env()

    # final_result aggregates all the results from all the projects.
    final_result = manager_runner.run_from_project_list_xlsx(projects_xlsx, enable_scaling_study)
    print(final_result)

    # returns an exit code of either 0 (successful validation), or 1 (validation failed).
    #
    # If validation was not enabled, exit with a status of 0 (no errors)

    if validation_enabled:
        exit(build_status)
    else:
        exit(0)


>>>>>>>> Begin run 2022-06-07 16:05:00.815155 <<<<<<<<<<
>>> Project and parametric lists loaded
<><><><><><><><><><><><><><><><><><> ge15_dist_165 <><><><><><><><><><><><><><><><><><>
>>> project_id: ge15_dist_165
>>> Project data: ~/Desktop/project_data/ge15_public.xlsx
{'F_dead_kN_per_turbine': 1798.08695652174, 'F_horiz_kN_per_turbine': 624.8822519784526, 'M_tot_kN_m_per_turbine': 47120.0, 'Radius_o_m': 7.365382452356683, 'Radius_s_m': 1.1917109249299527, 'Radius_g_m': 0, 'Radius_b_m': 7.164040491294037, 'Radius_m': 7.365382452356683, 'excavated_volume_m3': 458.6712992107442, 'foundation_volume_concrete_m3_per_turbine': 180.9943404451053, 'material_needs_per_turbine':     Material type ID  Quantity of material        Units
0      Steel - rebar             20.611635  ton (short)
1  Concrete 5000 psi            233.180574  cubic yards
2     Excavated dirt            599.919126  cubic_yards
3           Backfill            599.919126  cubic_yards, 'steel_mass_short_ton_per_turbine': 20